## Sentimant analysis

In [ ]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('../artifacts/datasets/books_with_categories.txt')
df

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NonFiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NonFiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5142,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Fiction
5143,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NonFiction
5144,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,NonFiction


In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [ ]:
def get_average_emotion(desc):
  sentences = desc.split('. ')
  predicted_emotions = classifier(sentences)
  sentence_emotion_scores = []

  for sentence_results in predicted_emotions:
      scores = {}
      for emotion_data in sentence_results:
          scores[emotion_data['label']] = emotion_data['score']
      sentence_emotion_scores.append(scores)

  emotions_df = pd.DataFrame(sentence_emotion_scores)


  return emotions_df.max()

In [ ]:
emotions_df_columns = ['isbn10','anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
emotions_df = pd.DataFrame(columns=emotions_df_columns)

for i in tqdm(range(df.shape[0])):
  emotion_scores = get_average_emotion(df.loc[i,'description'])
  emotions_df.loc[i,emotion_scores.index] = emotion_scores.values
  emotions_df.loc[i,'isbn10'] = df.loc[i,'isbn10']

In [ ]:
books_with_emotion_scores = df.merge(emotions_df,on='isbn10',how='left')
books_with_emotion_scores.to_csv('../artifacts/datasets/books_final.txt',index=False)

In [ ]:
books_with_emotion_scores= pd.read_csv('../artifacts/datasets/books_final.txt')
emotion_list = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
emotions = books_with_emotion_scores[emotion_list]

In [3]:
import plotly_express as px

top_emotions = emotions.apply(lambda x :x.argmax(),axis=1)
px.bar(top_emotions.value_counts())

In [4]:
df = books_with_emotion_scores

## Embedding

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
import pandas as pd
embedding_df = pd.read_csv('../artifacts/datasets/embeddings.csv')
embds = embedding_df.values
embds.shape

(5176, 384)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

raw_docs = TextLoader('../artifacts/datasets/tagged_descriptions.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1,chunk_overlap=0,separator="\n")
docs = text_splitter.split_documents(raw_docs)

Created a chunk of size 1170, which is longer than the specified 1
Created a chunk of size 1216, which is longer than the specified 1
Created a chunk of size 375, which is longer than the specified 1
Created a chunk of size 311, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 484, which is longer than the specified 1
Created a chunk of size 962, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 845, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1090, which is longer than the specified 1
Created a chunk of size 1191, which is longer than the specified 1
Created a chunk of size 306, which is longer than the specified 1
Create

In [8]:
from langchain_chroma import Chroma

book_db = Chroma(
    collection_name='BooksCollection',
    embedding_function=None,
)

In [9]:
book_db._collection.add(
    embeddings=embds,
    documents=[doc.page_content for doc in docs],
    ids=[f"doc_{i}" for i in range(len(docs))]
)

In [10]:
def get_similar_books(query, book_db = book_db, model = model, n_results=50):
    query_vec = model.encode([query],normalize_embeddings=True)[0]
    recommendations = book_db._collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )
    ids = list(int(i[1:14]) for i in recommendations['documents'][0])
    return df[df['isbn13'].isin(ids)]

In [11]:
df.head(2)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,tagged_description,simple_category,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,9780002005883 A NOVEL THAT READERS and critics...,NaN,Fiction,0.029641,0.338239,0.983973,0.949027,0.697846,0.956065,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,9780002261982 A new 'Christie for Christmas' -...,NaN,Fiction,0.594468,0.461991,0.935215,0.704422,0.891109,0.051414,0.212221


In [12]:
def get_book_of_category(cat:str,df:pd.DataFrame,n_result:int)->pd.DataFrame:
    if cat!= 'all':
        return df[df['simple_categories']==cat].head(n_result)
    else:
        return df.head(n_result)
    
def get_book_with_sentimance(sentimance:str,df:pd.DataFrame)-> pd.DataFrame:
    if sentimance =='all':
        return df
    elif sentimance == 'Happy':
        return df.sort_values(by='joy',ascending=False)
    elif sentimance == 'Surprising':
        return df.sort_values(by='surprise',ascending=False)
    elif sentimance == 'Angry':
        return df.sort_values(by='anger',ascending=False)
    elif sentimance == 'Suspenseful':
        return df.sort_values(by='fear',ascending=False)
    elif sentimance == 'Sad':
        return df.sort_values(by='sadness',ascending=False)

In [ ]:
def get_recommendations(query:str,category:str,emotion:str,n_results:int=10)->pd.DataFrame:
    similar_books = get_similar_books(query)
    books_of_category = get_book_of_category(category,similar_books,n_results)
    book_with_sentimance = get_book_with_sentimance(emotion,books_of_category)
    return book_with_sentimance
    
    Happy, Surprising, Angry, Suspensefull, Sad

In [30]:
b1 = get_recommendations('A story book about trying to love nature','all','all',5)

In [31]:
b1.title.values

array(['The Four Loves', 'The Family Way', 'The Paradox of Choice',
       'Charms for the Easy Life', 'Holy the Firm'], dtype=object)

In [32]:
b1.description.values

array(["Lewis' work on the nature of love divides love into four categories; Affection, Friendship, Eros and Charity. The first three come naturally to humanity. Charity, however, the Gift-love of God, is divine, and without this supernatural love, the natural loves become distorted and even dangerous.",
       "It should be the most natural thing in the world. But in Tony Parsons' latest bestseller, three couples discover that Mother Nature can be one hell of a bitch.",
       'The author of The Battle for Human Nature explains why too much choice has led to the ever increasing complexity of everyday decisions, why too much of a good thing has become detrimental to human psychological and emotional well-being, and how to focus our lives on making the right choices. Reprint. 35,000 first printing.',
       'A family without men, the Birches live gloriously offbeat lives in the lush, green backwoods of North Carolina. Radiant, headstrong Sophia and her shy, brilliant daughter, Margaret,

In [20]:
fiction_df = df[df['simple_categories'].isin(['Fiction',"Children's Fiction"])]

In [21]:
fiction_df.categories.value_counts()

categories
Fiction                                    2105
Juvenile Fiction                            390
Comics & Graphic Novels                     116
Drama                                        85
Poetry                                       51
                                           ... 
Conan (Fictitious character)                  1
Authors, Arab                                 1
Detective and mystery stories, American       1
Christian fiction                             1
Indic fiction (English)                       1
Name: count, Length: 101, dtype: int64